<a href="https://colab.research.google.com/github/ofer1992/notebooks/blob/main/bb_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [131]:
f = 1
n = 5*f + 1
propose = lambda: np.random.randint(1,3,size=(n-f,1))
propose()

array([[2],
       [1],
       [2],
       [1],
       [2]])

In [61]:
broadcast = lambda m: m.repeat(n-f, axis=1) # rows are senders, columns receivers
vote = lambda p: broadcast(p)
vote(propose())

array([[2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2],
       [2, 2, 2, 2, 2]])

In [100]:
# def commit(v):
v = vote(propose())
v = np.vstack([v, np.random.randint(1,3,size=(f,n-f))])
print(v)
def commit(v):
  counts_1 = (v == 1).sum(axis=0)
  counts_2 = (v == 2).sum(axis=0)
  # print(counts_1, counts_2)
  majority = np.where(counts_1 >= counts_2, 1, 2)
  majority_counts = np.where(counts_1 >= counts_2, counts_1, counts_2)
  commit = np.where(majority_counts >= n -f, majority, 0)
  # print(majority, majority_counts)
  # print(commit)
  return commit.reshape((-1,1))
commit(v)

[[1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 2 2 2 1 2 2 2 2 1 1 1]
 [2 1 2 1 2 2 1 1 2 1 2 1 1]
 [1 1 2 1 2 1 2 2 2 1 1 1 1]]


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [163]:
c = np.array([[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,0,0,1,0]])
print(c)
def blame(c):
  counts_0 = (c == 0).sum(axis=0)
  counts_1 = (c == 1).sum(axis=0)
  counts_2 = (c == 2).sum(axis=0)
  counts_minority = np.where(counts_1 < counts_2, counts_1, counts_2)
  possible_fakes = (f-counts_minority)
  possible_fakes[possible_fakes < 0] = 0
  return np.where(counts_0 > possible_fakes, 0, np.where(counts_1 >= counts_2, 1, 2)).reshape((-1,1))
print(blame(c))

[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 1 0]]
[[1]
 [1]
 [1]
 [1]
 [1]]


In [165]:
print(b)
def output(b, commited=-1):
  # print(b)
  counts_0 = (b == 0).sum(axis=0)
  # print(counts_0)
  return np.where(counts_0 >= f+1, 0, commited)
output(np.vstack([broadcast(b), np.random.randint(0,3,size=(f,n-f))]))

[[0]
 [0]
 [0]
 [0]
 [0]]


array([0, 0, 0, 0, 0])

In [166]:
for i in range(2000):
  p = propose()
  v = vote(p)
  v = np.vstack([v, np.random.randint(1,3,size=(f,n-f))])
  c = broadcast(commit(v))
  c = np.vstack([c, np.random.randint(0,3,size=(f,n-f))])
  b = blame(c)
  commited = np.max(b)
  b_broadcast = np.vstack([broadcast(b), np.random.randint(0,3,size=(f,n-f))])
  o = output(b_broadcast)
  # if not np.all(b == 0):
  if len(np.unique(o)) > 1:
    print("failure")
    print(f"p:\n{p}")
    print(f"v:\n{v}")
    print(f"c:\n{c}")
    print(f"b:\n{b_broadcast}")
    print(np.unique(b, return_counts=True))
    print(o)
    print()

failure
p:
[[1]
 [1]
 [2]
 [1]
 [1]]
v:
[[1 1 1 1 1]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 2 1 1]]
c:
[[1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 0 1]]
b:
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [1 1 1 1 1]
 [0 1 2 2 1]]
(array([0, 1]), array([1, 4]))
[ 0 -1 -1 -1 -1]

failure
p:
[[1]
 [1]
 [2]
 [1]
 [1]]
v:
[[1 1 1 1 1]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 2 1 1 1]]
c:
[[1 1 1 1 1]
 [0 0 0 0 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 0]]
b:
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 0 0 0 0]
 [0 0 1 0 2]]
(array([0, 1]), array([1, 4]))
[ 0  0 -1  0 -1]

failure
p:
[[2]
 [2]
 [1]
 [2]
 [2]]
v:
[[2 2 2 2 2]
 [2 2 2 2 2]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 1 2 2 2]]
c:
[[2 2 2 2 2]
 [0 0 0 0 0]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 0 2 2]]
b:
[[2 2 2 2 2]
 [2 2 2 2 2]
 [0 0 0 0 0]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 1 2 0 0]]
(array([0, 2]), array([1, 4]))
[-1 -1 -1  0  0]

fail

In [140]:
len(np.unique([1,2]))

2